# cancer数据下载

In [12]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

edge_options = EdgeOptions()
edge_options.add_argument("--window-size=1920,1080")
# edge_options.add_argument("start-maximized")

# 设置webdriver路径，例如使用ChromeDriver
driver = webdriver.Edge(options=edge_options)

# 打开网页
driver.get('https://gco.iarc.who.int/today/en/dataviz/bars?mode=population')

# 等待页面加载
time.sleep(35)


In [13]:
# 点击空白处
driver.find_element(By.XPATH, '/html/body').click()

# 点击与mode_population相关联的label标签
mode_population = driver.find_element(By.XPATH, "//label[@for='mode_population']")
mode_population.click()
# print(mode_population.is_displayed())
# print("success mode_population")

### 点击Measure选项

In [14]:
# 点击Measure选项

measure_sort = ['incidence', 'mortality']

incidence = driver.find_element(By.XPATH, "//label[@for='input_inc']")
mortality = driver.find_element(By.XPATH, "//label[@for='input_mort']")
# inc_mort = driver.find_element(By.XPATH, "//label[@for='input_inc_mort']")

def select_measure(measure):
    if measure is measure_sort[0]:
        incidence.click()
    elif measure is measure_sort[1]:
        mortality.click()
    else:
        driver.quit()
        exit()

### 选择cancer类型

In [15]:
# 选择cancer类型

cancer_sort = ['Melanoma of skin', 'Non-melanoma skin cancer']

def select_cancer(cancer_name):
    cancer_label = driver.find_element(By.XPATH, "//*[@id='tab_1']/fieldset[5]/div/div[2]/span")
    cancer_label.click()

    time.sleep(1)

    cancer_input = driver.find_element(By.ID, "multiselect-cancer")

    cancer_input.send_keys(cancer_name)
    time.sleep(1)
    cancer_input.send_keys(Keys.DOWN)
    cancer_input.send_keys(Keys.ENTER)

    time.sleep(3)

### 选择地区

In [16]:
# 选择地区

region_sort = ['Asia', 'Africa', 'Europe', 'Northern America', 'Latin America and the Caribbean', 'Oceania']

def select_region(region):
    region_label = driver.find_element(By.XPATH, "//*[@id='tab_1']/fieldset[6]/div/div[2]/span")
    region_label.click()

    time.sleep(1)

    region_input = driver.find_element(By.XPATH, "//*[@id='tab_1']/fieldset[6]/div/div[2]/input")
    region_input.send_keys(region)
    region_input.send_keys(Keys.ENTER)

    time.sleep(3)

### 选择年龄段

In [17]:
# 选择年龄段

left_sort = [0, 2, 4, 6, 8, 10, 12, 14]
right_sort = [1, 3, 5, 7, 9, 11, 13, 15]

# left_list = WebDriverWait(driver, 10).until(
#     EC.visibility_of_element_located((By.XPATH, '//*[@id="tab_1"]/fieldset[8]/div[1]/div[3]'))
# )

# right_list = WebDriverWait(driver, 10).until(
#     EC.visibility_of_element_located((By.XPATH, '//*[@id="tab_1"]/fieldset[8]/div[2]/div[3]'))
# )

def select_age(index):
    # 年龄下限
    left_age_label = driver.find_element(By.XPATH, "//*[@id='tab_1']/fieldset[8]/div[1]/div[2]/span")
    left_age_label.click()

    time.sleep(1)

    left_age_xpath = f"//*[@id='tab_1']/fieldset[8]/div[1]/div[3]/ul[1]/li[@id='null-{left_sort[index]}']"
    left_age = driver.find_element(By.XPATH, left_age_xpath)
    left_age.click()

    time.sleep(1)

    # 年龄上限
    right_age_label = driver.find_element(By.XPATH, "//*[@id='tab_1']/fieldset[8]/div[2]/div[2]/span")
    right_age_label.click()

    time.sleep(1)

    right_age_xpath = f"//*[@id='tab_1']/fieldset[8]/div[2]/div[3]/ul[1]/li[@id='null-{right_sort[index]}']"
    right_age = driver.find_element(By.XPATH, right_age_xpath)
    right_age.click()

    # 年龄下限
    left_age_label.click()
    time.sleep(1)
    left_age.click()

    time.sleep(10)

### 选择download

In [18]:
# 选择download

def click_download():
    download_label = driver.find_element(
        By.XPATH, 
        "//*[@id='app']/div/div[2]/div[2]/div/div[2]/div[2]/div[1]/ul[1]/li[2]"
    )
    download_label.click()

    time.sleep(3)

### 下载json文件, 并重命名

In [19]:
# 下载json文件, 并重命名

region_map = {
    'Asia': 'asia', 
    'Africa': 'africa', 
    'Europe': 'europe', 
    'Northern America': 'na', 
    'Latin America and the Caribbean': 'lac',
    'Oceania': 'oceania'
}

measure_map = {
    'incidence': 'incidence',
    'mortality': 'mortality',
}

cancer_map = {
    'Melanoma of skin': 'melanoma',
    'Non-melanoma skin cancer': 'nomelanoma'
}

age_map = [10, 20, 30, 40, 50, 60, 70, 80]

def download_json(region, measure, cancer, age_index):
    json_label = driver.find_element(By.XPATH, "//*[@id='tab_downloads']/div/div/div[2]/ul[3]/li/a")

    filename = f"{region_map[region]}_{measure_map[measure]}_{cancer_map[cancer]}_{age_map[age_index]}"
    driver.execute_script(f"arguments[0].setAttribute('download', '{filename}')", json_label)
    time.sleep(2)
    json_label.click()
    time.sleep(3)
    print(f"success: {filename}.json")

### 主函数

In [21]:
# 主函数

dir = "C:\\Users\\zcj\\Downloads"

def main(region, measure, cancer, age_index):
    filename = f"{region_map[region]}_{measure_map[measure]}_{cancer_map[cancer]}_{age_map[age_index]}.json"
    file_path=os.path.join(dir, filename)
    if os.path.exists(file_path):
        return

    select_age(age_index)
    click_download()
    download_json(region, measure, cancer, age_index)

for measure in measure_sort:
    select_measure(measure)
    for cancer in cancer_sort:
        select_cancer(cancer)
        for region in region_sort:
            select_region(region)
            for index in range(8):
                main(region, measure, cancer, index)

success: asia_incidence_nomelanoma_80.json
success: africa_incidence_nomelanoma_10.json
success: africa_incidence_nomelanoma_20.json
success: africa_incidence_nomelanoma_30.json
success: africa_incidence_nomelanoma_40.json
success: africa_incidence_nomelanoma_50.json
success: africa_incidence_nomelanoma_60.json
success: africa_incidence_nomelanoma_70.json
success: africa_incidence_nomelanoma_80.json
success: europe_incidence_nomelanoma_10.json
success: europe_incidence_nomelanoma_20.json
success: europe_incidence_nomelanoma_30.json
success: europe_incidence_nomelanoma_40.json
success: europe_incidence_nomelanoma_50.json
success: europe_incidence_nomelanoma_60.json
success: europe_incidence_nomelanoma_70.json
success: europe_incidence_nomelanoma_80.json
success: na_incidence_nomelanoma_10.json
success: na_incidence_nomelanoma_20.json
success: na_incidence_nomelanoma_30.json
success: na_incidence_nomelanoma_40.json
success: na_incidence_nomelanoma_50.json
success: na_incidence_nomelanoma

In [ ]:
driver.quit()